In [1]:
import numpy as np
import pandas as pd
import tempfile

from collections import Counter
from dataclasses import dataclass
from tqdm.notebook import tqdm

from datapaths import *

pd.set_option('display.max_rows', 4)
pd.set_option('display.max_columns', None)


In [2]:
flipon_to_data = pd.read_table(F_FLIPON_TO_DATA)
flipon_to_data

,Flipon,Coordinates,Strand,Gene Feature,Gene Name,Gene Strand,Gene Type,Gene ID,Transcript ID,c. M miR (+),c. M miR (-),c. MJ miR (+),c. MJ miR (-),c. J miR (+),c. J miR (-),all M miR (+),all M miR (-),all MJ miR (+),all MJ miR (-),all J miR (+),all J miR (-),cCRE (+-200bp),LINE/LTR (+-200bp),Feature Group
0,g4,chr1:3014794-3014871,-,Distal Intergenic,4933401J01Rik,+,TEC,ENSMUSG00000102693.1,ENSMUST00000193812.1,miR-328 (1),NaN,miR-486 (1),NaN,NaN,NaN,"miR-6994 (1), miR-129b (1)","miR-92a-2 (1), miR-698/miR-7078 (1), miR-3572 ...",miR-486a/miR-486b (1),"miR-3076 (1), miR-6418 (1), miR-6987 (1)",NaN,miR-7649 (1),NaN,"LINE@L1Md_F2(-), LINE@L1VL1(+)",LINE/LTR
1,g4,chr1:3099888-3099963,+,Promoter (2-3kb),Gm26206,+,snRNA,ENSMUSG00000064842.1,ENSMUST00000082908.1,NaN,miR-328 (1),NaN,NaN,NaN,NaN,"miR-365-1/miR-365-2 (1), miR-698/miR-7078 (1),...","miR-221 (1), miR-129b (1)","miR-6418 (1), miR-7067 (1)",NaN,NaN,NaN,NaN,"LINE@L1Md_F2(+), LTR@MTC(+)",LINE/LTR
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77706,z-dna,chrY:4203100-4203146,.,Distal Intergenic,Gm28191,+,unprocessed_pseudogene,ENSMUSG00000100300.1,ENSMUST00000189112.1,NaN,NaN,miR-493 (1),NaN,NaN,NaN,"miR-466d/miR-466i (4), miR-466l (5)",NaN,miR-493 (1),NaN,NaN,NaN,NaN,LINE@L1_Mur1(-),LINE/LTR
77707,z-dna,chrY:4207093-4207113,.,Distal Intergenic,Gm28191,+,unprocessed_pseudogene,ENSMUSG00000100300.1,ENSMUST00000189112.1,NaN,NaN,NaN,NaN,NaN,NaN,"miR-466a/miR-466e/miR-466p (1), miR-466d/miR-4...",NaN,NaN,NaN,NaN,NaN,NaN,"LINE@L1_Mus3(+), LINE@L1Md_A(+), LTR@ORR1B1(+)",LINE/LTR


In [42]:
# print('Conserved miR, M')

# result = pd.DataFrame()

# plus = flipon_to_data[flipon_to_data['c. miR (+)'].notna() & flipon_to_data['c. miR (-)'].isna()]['Flipon'].value_counts().sort_index()
# result = pd.concat([result, pd.Series(plus.values, index=plus.index, name='(+)')], axis=1)

# minus = flipon_to_data[flipon_to_data['c. miR (+)'].isna() & flipon_to_data['c. miR (-)'].notna()]['Flipon'].value_counts().sort_index()
# result = pd.concat([result, pd.Series(minus.values, index=minus.index, name='(-)')], axis=1)

# plus_or_minus = flipon_to_data[flipon_to_data['c. miR (+)'].notna() | flipon_to_data['c. miR (-)'].notna()]['Flipon'].value_counts().sort_index()
# result = pd.concat([result, pd.Series(plus_or_minus.values, index=plus_or_minus.index, name='(+) or (-)')], axis=1)

# plus_and_minus = flipon_to_data[flipon_to_data['c. miR (+)'].notna() & flipon_to_data['c. miR (-)'].notna()]['Flipon'].value_counts().sort_index()
# result = pd.concat([result, pd.Series(plus_and_minus.values, index=plus_and_minus.index, name='(+) and (-)')], axis=1)

# display(result)


# print('Conserved miR, J')

# result = pd.DataFrame()

# plus = flipon_to_data[flipon_to_data['c. J miR (+)'].notna() & flipon_to_data['c. J miR (-)'].isna()]['Flipon'].value_counts().sort_index()
# result = pd.concat([result, pd.Series(plus.values, index=plus.index, name='(+)')], axis=1)

# minus = flipon_to_data[flipon_to_data['c. J miR (+)'].isna() & flipon_to_data['c. J miR (-)'].notna()]['Flipon'].value_counts().sort_index()
# result = pd.concat([result, pd.Series(minus.values, index=minus.index, name='(-)')], axis=1)

# plus_or_minus = flipon_to_data[flipon_to_data['c. J miR (+)'].notna() | flipon_to_data['c. J miR (-)'].notna()]['Flipon'].value_counts().sort_index()
# result = pd.concat([result, pd.Series(plus_or_minus.values, index=plus_or_minus.index, name='(+) or (-)')], axis=1)

# plus_and_minus = flipon_to_data[flipon_to_data['c. J miR (+)'].notna() & flipon_to_data['c. J miR (-)'].notna()]['Flipon'].value_counts().sort_index()
# result = pd.concat([result, pd.Series(plus_and_minus.values, index=plus_and_minus.index, name='(+) and (-)')], axis=1)

# display(result)


print('Conserved miR, MJ')

result = pd.DataFrame()

plus = flipon_to_data[flipon_to_data['c. MJ miR (+)'].notna() & flipon_to_data['c. MJ miR (-)'].isna()]['Flipon'].value_counts().sort_index()
result = pd.concat([result, pd.Series(plus.values, index=plus.index, name='(+)')], axis=1)

minus = flipon_to_data[flipon_to_data['c. MJ miR (+)'].isna() & flipon_to_data['c. MJ miR (-)'].notna()]['Flipon'].value_counts().sort_index()
result = pd.concat([result, pd.Series(minus.values, index=minus.index, name='(-)')], axis=1)

plus_or_minus = flipon_to_data[flipon_to_data['c. MJ miR (+)'].notna() | flipon_to_data['c. MJ miR (-)'].notna()]['Flipon'].value_counts().sort_index()
result = pd.concat([result, pd.Series(plus_or_minus.values, index=plus_or_minus.index, name='(+) or (-)')], axis=1)

plus_and_minus = flipon_to_data[flipon_to_data['c. MJ miR (+)'].notna() & flipon_to_data['c. MJ miR (-)'].notna()]['Flipon'].value_counts().sort_index()
result = pd.concat([result, pd.Series(plus_and_minus.values, index=plus_and_minus.index, name='(+) and (-)')], axis=1)

display(result)

Conserved miR, M


,(+),(-),(+) or (-),(+) and (-)
g4,3502,3430,7999,1067
h-dna,1346,1347,2821,128
sidd,2478,2582,14725,9665
z-dna,2797,2754,6145,594


Conserved miR, J


,(+),(-),(+) or (-),(+) and (-)
g4,709,701,1412,2
h-dna,426,425,854,3
sidd,425,477,905,3
z-dna,930,818,1761,13


Conserved miR, MJ


,(+),(-),(+) or (-),(+) and (-)
g4,3106,3229,6807,472
h-dna,1649,1636,3432,147
sidd,1810,1839,3945,296
z-dna,3617,3910,8005,478


In [43]:
print('All miR, M')

result = pd.DataFrame()

plus = flipon_to_data[flipon_to_data['miR (+)'].notna() & flipon_to_data['miR (-)'].isna()]['Flipon'].value_counts().sort_index()
result = pd.concat([result, pd.Series(plus.values, index=plus.index, name='(+)')], axis=1)

minus = flipon_to_data[flipon_to_data['miR (+)'].isna() & flipon_to_data['miR (-)'].notna()]['Flipon'].value_counts().sort_index()
result = pd.concat([result, pd.Series(minus.values, index=minus.index, name='(-)')], axis=1)

plus_or_minus = flipon_to_data[flipon_to_data['miR (+)'].notna() | flipon_to_data['miR (-)'].notna()]['Flipon'].value_counts().sort_index()
result = pd.concat([result, pd.Series(plus_or_minus.values, index=plus_or_minus.index, name='(+) or (-)')], axis=1)

plus_and_minus = flipon_to_data[flipon_to_data['miR (+)'].notna() & flipon_to_data['miR (-)'].notna()]['Flipon'].value_counts().sort_index()
result = pd.concat([result, pd.Series(plus_and_minus.values, index=plus_and_minus.index, name='(+) and (-)')], axis=1)

display(result)


print('All miR, J')

result = pd.DataFrame()

plus = flipon_to_data[flipon_to_data['J miR (+)'].notna() & flipon_to_data['J miR (-)'].isna()]['Flipon'].value_counts().sort_index()
result = pd.concat([result, pd.Series(plus.values, index=plus.index, name='(+)')], axis=1)

minus = flipon_to_data[flipon_to_data['J miR (+)'].isna() & flipon_to_data['J miR (-)'].notna()]['Flipon'].value_counts().sort_index()
result = pd.concat([result, pd.Series(minus.values, index=minus.index, name='(-)')], axis=1)

plus_or_minus = flipon_to_data[flipon_to_data['J miR (+)'].notna() | flipon_to_data['J miR (-)'].notna()]['Flipon'].value_counts().sort_index()
result = pd.concat([result, pd.Series(plus_or_minus.values, index=plus_or_minus.index, name='(+) or (-)')], axis=1)

plus_and_minus = flipon_to_data[flipon_to_data['J miR (+)'].notna() & flipon_to_data['J miR (-)'].notna()]['Flipon'].value_counts().sort_index()
result = pd.concat([result, pd.Series(plus_and_minus.values, index=plus_and_minus.index, name='(+) and (-)')], axis=1)

display(result)


print('All miR, MJ')

result = pd.DataFrame()

plus = flipon_to_data[flipon_to_data['MJ miR (+)'].notna() & flipon_to_data['MJ miR (-)'].isna()]['Flipon'].value_counts().sort_index()
result = pd.concat([result, pd.Series(plus.values, index=plus.index, name='(+)')], axis=1)

minus = flipon_to_data[flipon_to_data['MJ miR (+)'].isna() & flipon_to_data['MJ miR (-)'].notna()]['Flipon'].value_counts().sort_index()
result = pd.concat([result, pd.Series(minus.values, index=minus.index, name='(-)')], axis=1)

plus_or_minus = flipon_to_data[flipon_to_data['MJ miR (+)'].notna() | flipon_to_data['MJ miR (-)'].notna()]['Flipon'].value_counts().sort_index()
result = pd.concat([result, pd.Series(plus_or_minus.values, index=plus_or_minus.index, name='(+) or (-)')], axis=1)

plus_and_minus = flipon_to_data[flipon_to_data['MJ miR (+)'].notna() & flipon_to_data['MJ miR (-)'].notna()]['Flipon'].value_counts().sort_index()
result = pd.concat([result, pd.Series(plus_and_minus.values, index=plus_and_minus.index, name='(+) and (-)')], axis=1)

display(result)

All miR, M


,(+),(-),(+) or (-),(+) and (-)
g4,6353,6384,19790,7053
h-dna,4618,4654,14061,4789
sidd,171,187,15294,14936
z-dna,7114,7092,16573,2367


All miR, J


,(+),(-),(+) or (-),(+) and (-)
g4,2015,2036,4332,281
h-dna,3068,3027,6516,421
sidd,2354,2360,5022,308
z-dna,2915,3112,6375,348


All miR, MJ


,(+),(-),(+) or (-),(+) and (-)
g4,7124,7223,18498,4151
h-dna,4939,4877,14347,4531
sidd,4187,4430,12471,3854
z-dna,6804,6407,17957,4746
